# Producing an Interative Dashboard to Monitor Cells Activity Across the City

### This code uses the raw dataset to identify cells activity across the city. Thereafter, by aggregating towers by traffic zones, we can identify which parts of the city are more crowded

In [ ]:
import dask.dataframe as dd
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from keplergl import KeplerGl
import geopandas as gpd
import matplotlib

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=6, 
                       threads_per_worker=2,
                       memory_limit='4GB')
client = Client(cluster)
client

# import datasets

In [ ]:
data = dd.read_csv(r'E:***\raw_datas.csv',parse_dates=['Date_Time'])

# filter data frames

In [ ]:
data['Hour'] = data.Date_Time.dt.hour
data_filteredbyH = data[(data.Hour>=6) & (data.Hour<=24)]
data_gbyH = data_filteredbyH.groupby('Hour')

# spatial Join

In [ ]:
def custom_join(df):
    shp1 = gpd.read_file(r'E:\Arshad\Thesis\Datasets\Shapefiles\Tehran Mahalat\mahalat.shp') #send this for mr Rahmani
    shp = shp1.to_crs({'init': 'epsg:4326'})
    geodf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(x=df.Long, y=df.Lat), crs={'init': 'epsg:4326'})
    joined_data = gpd.sjoin(geodf, shp, how='left',op='within')
    joined_data = pd.DataFrame(joined_data)
    output = joined_data.groupby('index_right')['REGION'].count().reset_index()
    
    return output

In [ ]:
countPerMahale = data_gbyH.apply(custom_join,meta={'index_right':np.int64,'REGION':np.int64}).compute()

# Plotting

In [ ]:
shp1 = gpd.read_file(r'E:\Arshad\Thesis\Datasets\Shapefiles\Tehran Mahalat\mahalat.shp')
shp = shp1.to_crs({'init': 'epsg:4326'})
Hours ={}
shp['index_right'] = range(0,353)

for i in range(6,24):
    Merged_shp = shp.merge(countPerMahale.loc[i],on='index_right',how='outer')
    Merged_shp.fillna(0,inplace=True)
    Hours['{}'.format(i)]=Merged_shp

In [ ]:
for i in range(6,24):
    plt.figure()
    Hours['{}'.format(i)].plot(figsize=(17,17), edgecolor="grey",cmap='OrRd',column='REGION_y',alpha=0.8)
    plt.title('Hour {}'.format(i))

In [ ]:
Hours['22'].plot(figsize=(17,17), edgecolor="grey",cmap='OrRd',column='REGION_y',alpha=0.8)

In [ ]:
#create dataframe for pandas bokeh
l=[]
for i in range(6,24):
    shp['Hour {} Activity'.format(i)] = Hours['{}'.format(i)]['REGION_y']
    l.append('Hour {} Activity'.format(i))
    

# Pandas Bokeh

In [ ]:
import pandas_bokeh
pandas_bokeh.output_notebook()

from bokeh.palettes import brewer
c = brewer['Reds'][8]
c = c[::-1]

In [ ]:
slider_columns = l
slider_range = range(6, 24)
shp.plot_bokeh(
    figsize=(900, 600),
    slider=slider_columns,
    slider_range=slider_range,
    slider_name="Hour", 
    colormap=c,
    hovertool_columns=["ENAME"] + slider_columns,
    title="No of Activity",alpha = 0.8)

# Prepairing the Kepler dataset

In [ ]:
#this is for keplergl
ll=[]
for i in range(6,24):
    ddf=Hours['{}'.format(i)] 
    ddf['Time'] = pd.to_datetime('2019 {}:00:00'.format(i))
    ll.append(ddf)
df_keplergl=pd.concat(ll).reset_index()
df_keplergl

df_keplergl.to_csv('E:file.csv',index=True)